In [2]:
import numpy as np
import h5py
import lmdb
import caffe
from eval.inference import infer_to_h5_fixed_dims
from iow import read_lmdb, to_lmdb
from iow.lmdb_utils import MAP_SZ, IDX_FMT
from blobs.mat_utils import expand_dims

In [3]:
def path_to(path):
    return base_path + path
base_path = '/mnt/scratch/pierre/caffe_sandbox_tryouts/'
fpath_net = path_to('learning_curve/prototxt/net0_test.prototxt')
fpath_weights = path_to('learning_curve/snapshots/net0_snapshot_iter_10000.caffemodel')
fpath_db = path_to('inference/mnist_%s_train_lmdb')
fpath_dst = path_to('inference/db.h5')

net = caffe.Net(fpath_net, fpath_weights, caffe.TEST) # or caffe.TRAIN
keys = ['accuracy']
n = 100

infer_to_h5_fixed_dims(net, keys, n, fpath_dst, preserve_batch=True)

[100]

In [62]:
for layer_name, layer_data in net.blobs.items():
    if layer_name <> 'accuracy' and layer_name <> 'loss' and layer_name <> 'data':
        print layer_name, layer_data.data[0]

label 8.0
label_data_1_split_0 8.0
label_data_1_split_1 8.0
fc1 [-1.23041511 -1.47406483  2.29490113 -2.00914955 -0.61503345  1.0857265
 -2.60216951 -3.27017665 -1.56690454  1.36369002  0.0062819   0.87156957
  1.87490773  2.15755987  2.20640016 -1.9203403  -0.76069725  0.22655986
 -1.32125497  3.28743386  1.5807426   1.11090732 -0.88687688  3.02278304
 -2.47363067  0.72018868  1.02886331 -1.25145483  1.6719054  -0.64129353
 -0.12661529 -3.61808348 -1.09233677 -2.00988102  0.77568561 -0.22447476
 -1.80748129 -2.67294502 -1.58403552 -0.23509668 -1.02196229 -1.35729921
 -0.76598269  1.76095688  0.08710995  1.40730906  0.0184609   1.48187971
  1.81837654  0.82379246]
sig1 [ 0.22610877  0.18632556  0.90845388  0.11824562  0.35091186  0.74757612
  0.06899893  0.0366086   0.17265812  0.79635876  0.50157046  0.70507216
  0.86702514  0.89637309  0.90082276  0.12782362  0.31849492  0.55639893
  0.21060957  0.96399522  0.82930964  0.75229824  0.29175475  0.95359284
  0.07772757  0.67264855  0.73

In [42]:
def hdf5_reader(file_name, key):
    f = h5py.File(file_name, 'r') # open read-only
    group = f[key]
    if isinstance(group, h5py.Dataset):
        yield group.value
    else:
        for key, value in dict(group).iteritems():
            datum = caffe.proto.caffe_pb2.Datum()
            datum.ParseFromString(value)
            label = int(datum.label)
            image = caffe.io.datum_to_array(datum).astype(np.uint8)
            yield key, flat_shape(image), label

for value in hdf5_reader('../inference/db.h5', 'accuracy'):
    print value

[ 0.97000003  0.98000002  0.94999999  0.89999998  0.92000002  0.88999999
  0.94        0.94999999  0.94999999  0.91000003  0.92000002  0.88
  0.85000002  0.92000002  0.91000003  0.88999999  0.88        0.88        0.94
  0.88        0.88999999  0.89999998  0.94        0.92000002  0.93000001
  0.93000001  0.93000001  0.94        0.93000001  0.89999998  0.97000003
  0.88999999  0.94        0.94999999  0.94        0.91000003  0.94
  0.88999999  0.82999998  0.88999999  0.91000003  0.93000001  0.89999998
  0.92000002  0.92000002  0.94        0.94999999  0.94999999  0.86000001
  0.94        0.93000001  0.95999998  0.98000002  0.99000001  0.95999998
  0.99000001  0.92000002  0.97000003  0.91000003  0.92000002  0.91000003
  0.94        1.          0.94        0.94999999  0.89999998  0.92000002
  0.89999998  0.99000001  0.97000003  1.          0.99000001  0.97000003
  0.99000001  0.94        0.97000003  1.          0.99000001  0.87
  0.93000001  0.92000002  0.99000001  0.94        0.98000002  0

In [ ]:
# accuracy, roc curves, precision recall, confusion matrix
from sklearn.metrics import accuracy_score, roc_curve, precision_recall_curve, confusion_matrix

accuracy_score(y_true, y_pred)
fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=2)
precision, recall, thresholds = precision_recall_curve(y_true, y_score)
confusion_matrix(y_true, y_pred, labels=[0., 1., 2., 3., 4., 5., 6., 7., 8., 9.]) # np.arange(0, 10, 1.)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()